# Model evaluation - Optimal thresholding on default probas
In this notebook, we use a LogReg trained on 1M SIRETs, evaluated on 250k SIRETs.<br>
We compute $f_{\beta}$ scores, balanced accuracy and select thresholds optimally.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Set logging level to INFO
import logging
logging.getLogger().setLevel(logging.INFO)

# Import required libraries and modules
from datetime import datetime
from pathlib import Path
import pickle
import json
from types import ModuleType

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, fbeta_score, average_precision_score
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper

from predictsignauxfaibles.config import IGNORE_NA
from predictsignauxfaibles.data import SFDataset
from predictsignauxfaibles.evaluate import evaluate, make_precision_recall_curve, make_thresholds_from_fbeta, make_thresholds_from_conditions
from predictsignauxfaibles.merge_models import merge_models
from predictsignauxfaibles.pipelines import run_pipeline
from predictsignauxfaibles.utils import assign_flag, load_conf, log_splits_size

### Loading predictions from csv, splitting models

In [ ]:
## LOADING MODELS FROM FILES

model_runs = {
    "default": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/default_20210531-165745",
    "small": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/small_20210531-180946"
}

## DEFAULT MODEL PIPELINE

default = pd.read_csv(f"{model_runs['default']}/predictions.csv")
default_mapper_unpickled =  pickle.load(
    open( f"{model_runs['default']}/model_comp0.pickle",
        "rb"
        )
)
default_mapper = default_mapper_unpickled[1]
default_model_unpickled = pickle.load(
    open( f"{model_runs['default']}/model_comp1.pickle",
        "rb"
        )
)
default_model = default_model_unpickled[1]

default_pp = Pipeline(
    [("transform_dataframe", default_mapper), ("fit_model", default_model)]
)

## SMALL MODEL PIPELINE

small = pd.read_csv(f"{model_runs['small']}/predictions.csv")
small_mapper_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp0.pickle",
        "rb"
        )
)
small_mapper = small_mapper_unpickled[1]
small_model_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp1.pickle",
        "rb"
        )
)
small_model = small_model_unpickled[1]

small_pp = Pipeline(
    [("transform_dataframe", small_mapper), ("fit_model", small_model)]
)

### Threshold selection for default model

In [ ]:
default_conf = load_conf(model_name="default")
test = default_conf.TEST_DATASET
test.sample_size = 2.5e5

In [ ]:
#test.fetch_data()
test.data = pd.read_csv("/home/common/benchmark/052021_full_data_test.csv") #"/home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv"

In [ ]:
test.replace_missing_data().remove_na(ignore=IGNORE_NA)
test.data = run_pipeline(test.data, default_conf.TRANSFO_PIPELINE)

In [ ]:
test_features_unmapped = test.data[set(test.data.columns).difference(set(["outcome"]))]
test_outcomes = test.data["outcome"].astype(int).to_numpy()
test_features = default_mapper.transform(test_features_unmapped)

Isolating transformed features on one hand, outcomes on the other:

A couple of quicks checks/diagnoses:

In [ ]:
default.shape

In [ ]:
small.shape

In [ ]:
test.data.columns

In [ ]:
test.data.sample(n=3)

In [ ]:
100*test.data.groupby(by="outcome").siret.count()/len(test)

### Computing precision/recall graph as a function of classification thresholds - Default model

Let's load a test set from which we can use the output to measure our performance

In [ ]:
precision, recall, thresh = make_precision_recall_curve(
    test,
    default_pp
)

In [ ]:
precision

### Computing AUCPR
Computing the Area Under Curve fopr Precision-Recall curve (AUCPR), a metric which summarizes the potential of the model itself, without specifying the hyperparameter tuning that must be led to weight the relative importance of  precision and recall.

In [ ]:
aucpr = average_precision_score(test_outcomes, default_model.predict_proba(test_features)[:, 1])

In [ ]:
aucpr

### Option 1: Determine the thresholds by hand, looking at the Type2-Type1 errors plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

type_2 = 1 - recall
type_1 = 1 - precision

ax.scatter(
    type_1,
    type_2,
    label = "logreg"
);

plt.axhline(y=0.415, color='orange', linestyle='-')
plt.axvline(x=0.09, color='red', linestyle='-')

ax.set_xlabel('Type 1 error', fontsize=16)
ax.set_ylabel('Type 2 error', fontsize=16)

ax.legend(fontsize=18);

On peut sélectionner un seuil F1 (rouge) et un seuil F2 (orange) à partir de cette courbe:
- pour la liste F1, on cherche à maximiser la précision, puisqe la liste en question doit être la plus pertinent possible et contenir des entreprises "prioritaires". Cette liste minimise donc les faux positifs, favorise la précision et minimise donc l'erreur de type 1.
- la liste F2 doit se charger de minimiser les faux négatifs, et est donc plus orienter sur la maximisation du recall. Il s'agit ici de minimiser l'erreur de type 2.

Pour $t_{F1}$, on choisit une valeur qui donne une erreur de type 1 petite tout en étant la plus indulgente possible pour l'erreur de type 2. C'est typiquement ce qu'un point de courbure maximale va réaliser. Ici $t_{F1}=0.07$ semble assez satisfaisant (ligne verticale rouge ci-dessus).

Pour $t_{F2}$, on choisir au contraire une valeur de courbe (négative) minimale, tout en satisfaisant déjà une erreur de type 2 suffisament faible. Ici $t_{F2}=0.37$ semble assez satisfaisant (ligne horizontale orange ci-dessus).

### Option 2: Select thresholds by maximising a f-beta (with beta specified for each alert level)

In [ ]:
(t_F1, t_F2) = make_thresholds_from_fbeta(
    test_features,
    test_outcomes,
    default_model,
    beta_F1 = 0.5,
    beta_F2 = 2,
    n_thr = 1000
)

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $\beta=0.5$ - Optimal threshold: $t_{F1}=0.876$ - $f_{0.5}=0.683$<br>
F2 - $\beta=2$ - Optimal threshold: $t_{F2}=0.220$ - $f_{2}=0.495$

In [ ]:
evaluate(
    default_pp,
    test,
    beta=2,
    thresh=0.18
)

### Option 3: Select threshold by specifying a minimal precision for T1 alerts and a minimal recall for T1 + T2 alerts

In [ ]:
(t_F1_c, t_F2_c) = make_thresholds_from_conditions(
    precision,
    recall,
    thresh,
    min_precision_F1 = 1-0.09,
    min_recall_F2 = 1-0.415,
)

In [ ]:
t_F1_c

In [ ]:
t_F2_c

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $t_{F1}=0.921$ garantit une précision de 93% pour la liste F1 (rouge)<br>
F2 - $t_{F2}=0.124$ garantit un recall de 63% pour la liste F2 (orange)

### Threshold selection for small model

In [ ]:
small_conf = load_conf(model_name="small")
test = small_conf.TEST_DATASET
test.sample_size = 2.5e5

In [ ]:
#test.fetch_data()
test.data = pd.read_csv("/home/common/benchmark/052021_full_data_test.csv") #"/home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv"

In [ ]:
test.replace_missing_data().remove_na(ignore=IGNORE_NA)
test.data = run_pipeline(test.data, small_conf.TRANSFO_PIPELINE)

In [ ]:
test_features_unmapped = test.data[set(test.data.columns).difference(set(["outcome"]))]
test_outcomes = test.data["outcome"].astype(int).to_numpy()
test_features = small_mapper.transform(test_features_unmapped)

Isolating transformed features on one hand, outcomes on the other:

In [ ]:
test.data.columns

In [ ]:
test.data.sample(n=3)

In [ ]:
100*test.data.groupby(by="outcome").siret.count()/len(test)

### Computing precision/recall graph as a function of classification thresholds - Default model

Let's load a test set from which we can use the output to measure our performance

In [ ]:
precision, recall, thresh = make_precision_recall_curve(
    test,
    small_pp
)

In [ ]:
precision

### Computing AUCPR
Computing the Area Under Curve fopr Precision-Recall curve (AUCPR), a metric which summarizes the potential of the model itself, without specifying the hyperparameter tuning that must be led to weight the relative importance of  precision and recall.

In [ ]:
aucpr = average_precision_score(test_outcomes, small_model.predict_proba(test_features)[:, 1])

In [ ]:
aucpr

### Option 1: Determine the thresholds by hand, looking at the Type2-Type1 errors plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

type_2 = 1 - recall
type_1 = 1 - precision

ax.scatter(
    type_1,
    type_2,
    label = "logreg"
);

plt.axhline(y=0.415, color='orange', linestyle='-')
plt.axvline(x=0.09, color='red', linestyle='-')

ax.set_xlabel('Type 1 error', fontsize=16)
ax.set_ylabel('Type 2 error', fontsize=16)

ax.legend(fontsize=18);

On peut sélectionner un seuil F1 (rouge) et un seuil F2 (orange) à partir de cette courbe:
- pour la liste F1, on cherche à maximiser la précision, puisqe la liste en question doit être la plus pertinent possible et contenir des entreprises "prioritaires". Cette liste minimise donc les faux positifs, favorise la précision et minimise donc l'erreur de type 1.
- la liste F2 doit se charger de minimiser les faux négatifs, et est donc plus orienter sur la maximisation du recall. Il s'agit ici de minimiser l'erreur de type 2.

Pour $t_{F1}$, on choisit une valeur qui donne une erreur de type 1 petite tout en étant la plus indulgente possible pour l'erreur de type 2. C'est typiquement ce qu'un point de courbure maximale va réaliser. Ici $t_{F1}=0.07$ semble assez satisfaisant (ligne verticale rouge ci-dessus).

Pour $t_{F2}$, on choisir au contraire une valeur de courbe (négative) minimale, tout en satisfaisant déjà une erreur de type 2 suffisament faible. Ici $t_{F2}=0.37$ semble assez satisfaisant (ligne horizontale orange ci-dessus).

### Option 2: Select thresholds by maximising a f-beta (with beta specified for each alert level)

In [ ]:
(t_F1, t_F2) = make_thresholds_from_fbeta(
    test_features,
    test_outcomes,
    small_model,
    beta_F1 = 0.5,
    beta_F2 = 2,
    n_thr = 1000
)

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $\beta=0.5$ - Optimal threshold: $t_{F1}=0.876$ - $f_{0.5}=0.683$<br>
F2 - $\beta=2$ - Optimal threshold: $t_{F2}=0.220$ - $f_{2}=0.495$

In [ ]:
evaluate(
    small_pp,
    test,
    beta=2,
    thresh=0.127
)

### Option 3: Select threshold by specifying a minimal precision for T1 alerts and a minimal recall for T1 + T2 alerts

In [ ]:
(t_F1_c, t_F2_c) = make_thresholds_from_conditions(
    precision,
    recall,
    thresh,
    min_precision_F1 = 1-0.09,
    min_recall_F2 = 1-0.415,
)

In [ ]:
t_F1_c

In [ ]:
t_F2_c

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $t_{F1}=0.921$ garantit une précision de 93% pour la liste F1 (rouge)<br>
F2 - $t_{F2}=0.124$ garantit un recall de 63% pour la liste F2 (orange)

## To go further - Can we build a threshold selection criterion based on the variations of recall % threhold?

In [ ]:
# Let's compute the first-order derivative of the type-2 error
fod_recall = (recall[:-1] - np.roll(recall[:-1],1))/(thresh - np.roll(thresh,1))
fod_recall = fod_recall[1:]

In [ ]:
plt.hist(fod_recall)

In [ ]:
plt.plot(thresh)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(
    thresh[:-81],
    fod_recall[:-80],
    label = "logreg"
);

In [ ]:
recalL_maxvar_id = np.argmin(fod_recall[:-80])
recall_maxvar = fod_recall[recalL_maxvar_id]

recall_maxvar_precision = precision[recalL_maxvar_id]
recall_maxvar_recall = recall[recalL_maxvar_id]
recall_maxvar_thresh = thresh[recalL_maxvar_id]

In [ ]:
recall_maxvar_thresh

In [ ]:
recall_maxvar_recall

In [ ]:
recall_maxvar_precision

For now, we'll stick to manual selection based on the two types of error above, or maybe based on the maximisation of Fbeta